# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [36]:
# imports
import os
import sys
import ollama
from openai import OpenAI
from dotenv import load_dotenv

In [14]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [15]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [16]:
system_prompt = "You are assistant who has to search and think about the given question and answer it"
system_prompt += "Show my question first and answer it"

user_prompt = "Here is my Qeustion"
user_prompt += question
user_prompt += "Answer format follows this"
user_prompt += f'Your question is {question} and my answer is ~~'

In [34]:
# Get gpt-4o-mini to answer, with streaming
def get_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )

    for chunk in stream:
        # delta: 기존 Token에서 변경된 부분
        delta = chunk.choices[0].delta
        # delta.content: 답변 부분
        text = chunk.choices[0].delta.content
        if delta and text:
            sys.stdout.write(text)  # 줄바꿈 없이 바로바로 출력
            sys.stdout.flush()

In [35]:
get_answer(question)

Your question is  
Please explain what this code does and why:  
`yield from {book.get("author") for book in books if book.get("author")}` 

and my answer is  
This code is a Python generator expression that yields values from a set of authors extracted from a collection of `books`. Here’s a breakdown of its components:

1. **Set comprehension**: The code `{book.get("author") for book in books if book.get("author")}` creates a set containing authors' names. It iterates through each `book` in the `books` collection (which is assumed to be a list of dictionaries) and uses `book.get("author")` to get the author’s name. The condition `if book.get("author")` ensures that only books with a valid author are included.

2. **Yield from**: The `yield from` statement allows a generator to yield all values from the resulting set. It effectively delegates the yielding to another iterable, which in this case is the set of authors.

The use of a set comprehension ensures that all authors are unique, 

In [39]:
# Get Llama 3.2 to answer
def get_answer(question):
    answer = ollama.chat(
        model=MODEL_LLAMA,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )

    return answer['message']['content']

In [38]:
get_answer(question)

'~~This code is used to iterate over a list of books (`books`) and yield the author of each book. The `if book.get("author")` condition filters out any books that do not have an "author" key.\n\nThe `yield from {book.get("author") for book in books if book.get("author")}` expression is using a feature of Python called "yield from". This allows us to delegate the iteration and yielding of values from one iterable to another, rather than having to manually iterate over each value and yield it ourselves.~~'